### Sample program for transportation optimization problem  

#### Import libraries  

In [ ]:
import numpy as np
import pandas as pd
import pulp

#### Parameters  

In [ ]:
csv_cost = 'transport_cost.csv'
csv_supply = 'transport_supply.csv'
csv_demand = 'transport_demand.csv'

#### Read data from CSV file  

In [ ]:
df_cost=pd.read_csv(csv_cost, delimiter=',', skiprows=0, header=0)
print(df_cost.shape)
print(df_cost.info())
display(df_cost)

In [ ]:
df_supply = pd.read_csv(csv_supply, delimiter=',', skiprows=0, header=0)
display(df_supply)
ser_supply = df_supply['supply']
ser_supply.index = df_supply['warehouse']
print(ser_supply)

In [ ]:
df_demand = pd.read_csv(csv_demand, delimiter=',', skiprows=0, header=0)
display(df_demand)
ser_demand = df_demand['demand']
ser_demand.index = df_demand['factory']
display(ser_demand)

#### Define problem using PuLP functions  

In [ ]:
problem = pulp.LpProblem('transport', pulp.LpMinimize)  # generate LpProblem object

In [ ]:
var_list = []  # set LPVariables
for i in range(df_cost.shape[0]):
    var_list.append(pulp.LpVariable('v{}'.format(i), lowBound=0))
df_cost['Var'] = var_list
# or
#df_cost['Var'] = [pulp.LpVariable('v{}'.format(i), lowBound=0) for i in range(df_cost.shape[0])]

In [ ]:
display(df_cost)

In [ ]:
problem += pulp.lpDot(df_cost['cost'], df_cost['Var'])  # add Object function

In [ ]:
for w in ser_supply.index:
    w_var = df_cost[ df_cost['from']==w ]
    problem += pulp.lpSum(w_var['Var']) <= ser_supply[w]  # add Constraints for supply

# or
#for k, v in df_cost.groupby('from'):
#    problem += pulp.lpSum(v['Var']) <= ser_supply[k]
#    print(k)  # debug
#    display(v)  # debug

In [ ]:
for f in ser_demand.index:
    f_var = df_cost[ df_cost['to']==f ]
    problem += pulp.lpSum(f_var['Var']) >= ser_demand[f]  # add Constraints for demand
    
# or
#for k, v in df_cost.groupby('to'):
#    problem += pulp.lpSum(v['Var']) >= ser_demand[f]
#    print(k)  # debug
#    display(v)  # debug

#### Check problem  

In [ ]:
print(problem)

#### Solve  

In [ ]:
results = problem.solve()
print(pulp.LpStatus[results])

#### Show the results  

In [ ]:
df_cost['Value'] = df_cost['Var'].apply(pulp.value)
display(df_cost[df_cost['Value'] > 0])

In [ ]:
print('Total cost:', np.sum(df_cost['cost']*df_cost['Value']))